# dynamical modelling

# little bit about biopython

https://biopython.org/DIST/docs/tutorial/Tutorial.html [1]
https://medium.com/geekculture/phylogenetic-trees-implement-in-python-3f9df96c0c32 [2]
https://github.com/risg99/Phylogenetic-trees-python [2]

In [ ]:
# Importing necessary libraries
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import AlignIO

In [ ]:
# Read the sequences and align
align = AlignIO.read('msa.phy','phylip')


In [ ]:
import plotly.express as px

In [ ]:
# Calculate the distance matrix
calculator = DistanceCalculator('identity')
distMatrix = calculator.get_distance(align)
px.imshow(distMatrix)

In [ ]:

# Creating a DistanceTreeConstructor object
constructor = DistanceTreeConstructor()

####################################################
					#  NJ
####################################################

# Construct the phlyogenetic tree using NJ algorithm
NJTree = constructor.nj(distMatrix)

# Draw the phlyogenetic tree
Phylo.draw(NJTree)

# Printing the phlyogenetic tree using terminal
Phylo.draw_ascii(NJTree)


### check more tutorials from other researchers, once you know the basic structure, then you can develop more complicated scenarios

# tutorial: something about bird migration

Reference:

Main tutorial: https://github.com/florist-notes/Data-Analysis/tree/master

Additional tutorials:
https://daniel-furman.github.io/Python-species-distribution-modeling/


In [ ]:
import pandas as pd

df = pd.read_csv('dyn_zero/bird_tracking.csv', sep=',')
df.head(5)

In [ ]:
import folium
import geopandas

In [ ]:
# Create point geometries
geometry = geopandas.points_from_xy(df.longitude, df.latitude)
geometry

In [ ]:
df.columns

In [ ]:

geo_df = geopandas.GeoDataFrame(
    df[['altitude', 'date_time', 'device_info_serial', 'direction', 'latitude',
       'longitude', 'speed_2d', 'bird_name']], geometry=geometry
)

geo_df.head()

In [ ]:
import geodatasets

In [ ]:
world = geopandas.read_file(geodatasets.get_path("naturalearth.land"))
df.bird_name.unique()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(24, 18))
world.plot(ax=ax, alpha=0.4, color="grey")
geo_df.plot(column="bird_name", ax=ax, legend=True)
plt.title("Bird migration")

In [ ]:
import plotly.express as px

In [ ]:
df.columns

In [ ]:
fig = px.line_3d(df, x='longitude', y='latitude', z='altitude',
              color='bird_name', width=1080, height=720, title='A Scatter Plot of the Latitude,  Longitude and Altitude of birds')
fig.show()

In [ ]:
import scipy

from scipy import signal

In [ ]:
fig = px.line_3d(df, x=signal.savgol_filter(df['longitude'],
                           10, # window size used for filtering
                           3), 
                 y=signal.savgol_filter(df['latitude'],
                           10, # window size used for filtering
                           3), 
                 z=signal.savgol_filter(df['altitude'],
                           10, # window size used for filtering
                           3),
              color='bird_name', width=1080, height=720, title='A Scatter Plot of the Latitude,  Longitude and Altitude of birds')
fig.show()

In [ ]:
data = df[['latitude', 'longitude']]

In [ ]:
# let's also do native folium

#create a map
this_map = folium.Map(prefer_canvas=True)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
data.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

In [ ]:
df['date'] = df.date_time.str.split(' ', expand=True)[0]

In [ ]:
df.columns

In [ ]:
px.scatter(df, x=signal.savgol_filter(df['speed_2d'],
                           10, # window size used for filtering
                           3), 
           y=signal.savgol_filter(df['altitude'],
                           10, # window size used for filtering
                           3), 
           animation_frame="date", 
           animation_group="device_info_serial",
           color="bird_name", 
           hover_name="bird_name",
           log_x=False, 
           range_x=[-100, 100],
           range_y=[-100, 100])

# something about distributions

In [ ]:
# functions and imports

from scipy.stats import gaussian_kde
from numpy import mean
from numpy import std
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from scipy import stats
import scikit_posthocs as sp
import statistics

def calc_curve(data):
    """Calculate probability density."""
    min_, max_ = data.min(), data.max()
    X = [min_ + i * ((max_ - min_) / 500) for i in range(501)]
    Y = gaussian_kde(data).evaluate(X)
    return(X, Y)

from plotly.offline import plot


In [ ]:
df.columns

In [ ]:
xdf = df.copy()
xdf.describe()

In [ ]:
xdf = xdf.dropna()
xdf.describe()

# Assignment
1. Find a strategy to replace NA values in the columns.
Repeat the steps given below.

In [ ]:
data1 = xdf['altitude']
data2 = xdf['speed_2d']
data3 = xdf['direction']

X1, Y1 = calc_curve(data1)
X2, Y2 = calc_curve(data2)
X3, Y3 = calc_curve(data3)

traces = []
traces.append({'x': X1, 'y': Y1, 'name': 'Altitude'})
traces.append({'x': X2, 'y': Y2, 'name': 'Speed'})
traces.append({'x': X3, 'y': Y3, 'name': 'Direction'})

plot({'data': traces})

In [ ]:
# normalize and replot
xdf['norm_alt']=(xdf['altitude']-xdf['altitude'].mean())/xdf['altitude'].std()
xdf['speed_2d_alt']=(xdf['speed_2d']-xdf['speed_2d'].mean())/xdf['speed_2d'].std()
xdf['direction_alt']=(xdf['direction']-xdf['direction'].mean())/xdf['direction'].std()

In [ ]:
data1 = xdf['norm_alt']
data2 = xdf['speed_2d_alt']
data3 = xdf['direction_alt']

X1, Y1 = calc_curve(data1)
X2, Y2 = calc_curve(data2)
X3, Y3 = calc_curve(data3)

traces = []
traces.append({'x': X1, 'y': Y1, 'name': 'Altitude'})
traces.append({'x': X2, 'y': Y2, 'name': 'Speed'})
traces.append({'x': X3, 'y': Y3, 'name': 'Direction'})

plot({'data': traces})

# think about hypotheses

In [ ]:
xdf.set_index('date_time', drop=True, append=False, inplace=True, verify_integrity=False)
xdf = xdf.sort_index()
xdf

In [ ]:
px.line(y = xdf['norm_alt'], x=xdf.index)

In [ ]:
nxdf = xdf[['norm_alt', 'speed_2d_alt', 'direction_alt']]
nxdf

In [ ]:
fig = px.histogram(x=nxdf['speed_2d_alt'])
fig.show()

In [ ]:
import numpy as np

In [ ]:
# let's fft this signal

SAMPLE_RATE = 240  # a few months
DURATION = 24  # hours

def generate_sine_wave(freq, sample_rate, duration):
    x = np.linspace(0, duration, sample_rate * duration, endpoint=False)
    frequencies = x * freq
    # 2pi because np.sin takes radians
    y = np.sin((2 * np.pi) * frequencies)
    return x, y

# Generate a 2 hertz sine wave that lasts for 5 seconds
x, y = generate_sine_wave(2, SAMPLE_RATE, DURATION)
plt.plot(x, y)
plt.show()

In [ ]:
_, nice_tone = generate_sine_wave(400, SAMPLE_RATE, DURATION)
_, noise_tone = generate_sine_wave(4000, SAMPLE_RATE, DURATION)
noise_tone = noise_tone * 0.3

mixed_tone = nice_tone + noise_tone

In [ ]:
normalized_tone = np.int16((mixed_tone / mixed_tone.max()) * 32767)

plt.plot(normalized_tone[:1000])
plt.show()

In [ ]:
from scipy.fft import fft, fftfreq

# Number of samples in normalized_tone
N = SAMPLE_RATE * DURATION

yf = fft(normalized_tone)
xf = fftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf))
plt.show()

In [ ]:
# Assignment

Convert these plots to plotly.

### apply to speed? no need really

In [ ]:
nxdf.columns

In [ ]:
px.histogram(x=nxdf.index, y=nxdf['speed_2d_alt'])

# Bonus: PDFs, curve fitting

In [ ]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from scipy.stats._continuous_distns import _distn_names
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Best holders
    best_distributions = []

    # Estimate distribution parameters from data
    for ii, distribution in enumerate([d for d in _distn_names if not d in ['levy_stable', 'studentized_range']]):

        print("{:>3} / {:<3}: {}".format( ii+1, len(_distn_names), distribution ))

        distribution = getattr(st, distribution)

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')
                
                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]
                
                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))
                
                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                best_distributions.append((distribution, params, sse))
        
        except Exception:
            pass

    
    return sorted(best_distributions, key=lambda x:x[2])

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf


In [ ]:
# Load data from statsmodels datasets
data =  nxdf['speed_2d_alt']

# Plot for comparison
plt.figure(figsize=(12,8))
ax = data.plot(kind='hist', bins=50, density=True, alpha=0.5, color=list(matplotlib.rcParams['axes.prop_cycle'])[1]['color'])

# Save plot limits
dataYLim = ax.get_ylim()

# Find best fit distribution
best_distibutions = best_fit_distribution(data, 200, ax)
best_dist = best_distibutions[0]

# Update plots
ax.set_ylim(dataYLim)
ax.set_title(u'Strength of connection.\n All Fitted Distributions')
ax.set_xlabel(u'Strength (P1*P2)')
ax.set_ylabel('Frequency')

# Make PDF with best params 
pdf = make_pdf(best_dist[0], best_dist[1])

# Display
plt.figure(figsize=(12,8))
ax = pdf.plot(lw=2, label='PDF', legend=True)
data.plot(kind='hist', bins=50, density=True, alpha=0.5, label='Data', legend=True, ax=ax)

param_names = (best_dist[0].shapes + ', loc, scale').split(', ') if best_dist[0].shapes else ['loc', 'scale']
param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_dist[1])])
dist_str = '{}({})'.format(best_dist[0].name, param_str)

ax.set_title(u'Strength with best fit distribution \n' + dist_str)
ax.set_xlabel(u'Speed')
ax.set_ylabel('Frequency')

In [ ]:
xdf.columns

In [ ]:
# Load data from statsmodels datasets
data =  xdf['speed_2d']

# Plot for comparison
plt.figure(figsize=(12,8))
ax = data.plot(kind='hist', bins=50, density=True, alpha=0.5, color=list(matplotlib.rcParams['axes.prop_cycle'])[1]['color'])

# Save plot limits
dataYLim = ax.get_ylim()

# Find best fit distribution
best_distibutions = best_fit_distribution(data, 200, ax)
best_dist = best_distibutions[0]

# Update plots
ax.set_ylim(dataYLim)
ax.set_title(u'Strength of connection.\n All Fitted Distributions')
ax.set_xlabel(u'Strength (P1*P2)')
ax.set_ylabel('Frequency')

# Make PDF with best params 
pdf = make_pdf(best_dist[0], best_dist[1])

# Display
plt.figure(figsize=(12,8))
ax = pdf.plot(lw=2, label='PDF', legend=True)
data.plot(kind='hist', bins=50, density=True, alpha=0.5, label='Data', legend=True, ax=ax)

param_names = (best_dist[0].shapes + ', loc, scale').split(', ') if best_dist[0].shapes else ['loc', 'scale']
param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_dist[1])])
dist_str = '{}({})'.format(best_dist[0].name, param_str)

ax.set_title(u'Strength with best fit distribution \n' + dist_str)
ax.set_xlabel(u'Speed')
ax.set_ylabel('Frequency')

# Assignment part 2

Create three line plots for nxdf variables in the same plot, x axis is time.

# bonus material (mathematical thinking)
Markov chains and models
https://es.wikipedia.org/wiki/Andr%C3%A9i_M%C3%A1rkov
https://ericmjl.github.io/essays-on-data-science/machine-learning/markov-models/

